# 下载bitfinex交易所历史K线数据

## 1.定义下载时间段和品种

In [1]:
import time

In [2]:
now = int(round(time.time() * 1000))
then = now - (1000 * 60 * 60 * 24 * 30) # 30 days ago
symbol = 'BTCUSD'
symbol_bitfinex = 't' + symbol
vt_symbol = symbol + '.BITFINEX'

## 2.运行下载函数

In [12]:
%run download_history_candles_fun.py

## 4.执行下载

In [14]:
t = asyncio.ensure_future(run(symbol_bitfinex, then))
asyncio.get_event_loop().run_until_complete(t)


Task exception was never retrieved
future: <Task finished coro=<run() done, defined at E:\MyProjects\vnpy\mydocs\download_history_candles_fun.py:16> exception=NameError("name 'then' is not defined")>
Traceback (most recent call last):
  File "E:\MyProjects\vnpy\mydocs\download_history_candles_fun.py", line 17, in run
    await log_historical_candles(symbol_bitfinex, then)
  File "E:\MyProjects\vnpy\mydocs\download_history_candles_fun.py", line 13, in log_historical_candles
    candles = await bfx.rest.get_public_candles(symbol_bitfinex, 0, then)
NameError: name 'then' is not defined


RuntimeError: This event loop is already running

In [5]:
import sys
sys.path.append("E:\\MyProjects\\bitfinex")
%run E:\\MyProjects\\bitfinex\\bfxapi\\examples\\rest\\get_public_data.py

RuntimeError: This event loop is already running

Task exception was never retrieved
future: <Task finished coro=<run() done, defined at E:\MyProjects\bitfinex\bfxapi\examples\rest\get_public_data.py:41> exception=ClientConnectorError(10061, "Connect call failed ('199.16.156.40', 443)")>
Traceback (most recent call last):
  File "e:\vnstudio\lib\site-packages\aiohttp\connector.py", line 822, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)
  File "e:\vnstudio\lib\asyncio\base_events.py", line 948, in create_connection
    raise exceptions[0]
  File "e:\vnstudio\lib\asyncio\base_events.py", line 935, in create_connection
    await self.sock_connect(sock, address)
  File "e:\vnstudio\lib\asyncio\selector_events.py", line 475, in sock_connect
    return await fut
  File "e:\vnstudio\lib\asyncio\selector_events.py", line 505, in _sock_connect_cb
    raise OSError(err, f'Connect call failed {address}')
ConnectionRefusedError: [Errno 10061] Connect call failed ('199.16.156.40', 443)

The above except

In [11]:
myloop = asyncio.get_event_loop()
asyncio.ensure_future(run(symbol_bitfinex, then), loop = myloop).add_done_callback(fn)


NameError: name 'fn' is not defined

Task exception was never retrieved
future: <Task finished coro=<run() done, defined at E:\MyProjects\vnpy\mydocs\download_history_candles_fun.py:16> exception=NameError("name 'then' is not defined")>
Traceback (most recent call last):
  File "E:\MyProjects\vnpy\mydocs\download_history_candles_fun.py", line 17, in run
    await log_historical_candles(symbol_bitfinex)
  File "E:\MyProjects\vnpy\mydocs\download_history_candles_fun.py", line 13, in log_historical_candles
    candles = await bfx.rest.get_public_candles(symbol_bitfinex, 0, then)
NameError: name 'then' is not defined


## 5.转存数据为vnpy格式

In [ ]:
def generate_bar_from_row(row, symbol, exchange):
    """"""
    bar = DbBarData()

    bar.symbol = symbol
    bar.exchange = exchange
    bar.interval = "1m"
    bar.open_price = row["open"]
    bar.high_price = row["high"]
    bar.low_price = row["low"]
    bar.close_price = row["close"]
    bar.volume = row["volume"]
    bar.datetime = row.name.to_pydatetime()
    bar.gateway_name = "DB"
    bar.vt_symbol = f"{symbol}.{exchange}"

    return bar

def download_minute_bar(vt_symbol):
    """下载某一合约的分钟线数据"""
    print(f"开始下载合约数据{vt_symbol}")
    symbol, exchange = vt_symbol.split(".")

    start = time()
    with DB.atomic():
        for ix, row in candles.iterrows():
            print(row.name)
            bar = generate_bar_from_row(row, symbol, exchange)
            DbBarData.replace(bar.__data__).execute()

    end = time()
    cost = (end - start) * 1000

    print(
        "合约%s的分钟K线数据下载完成%s - %s，耗时%s毫秒"
        % (symbol, df.index[0], df.index[-1], cost)
    )
download_minute_bar(vt_symbol)

## 5.保存下载数据到vnpy数据库中

In [ ]:
import os
import unittest
from copy import copy
from datetime import datetime, timedelta

from vnpy.trader.constant import Exchange, Interval
from vnpy.trader.database.database import Driver
from vnpy.trader.object import BarData, TickData
os.environ["VNPY_TESTING"] = "1"

profiles = {Driver.SQLITE: {"driver": "sqlite", "database": "mydb.db"}}
def now():
    return datetime.utcnow()


bar = DbBarData(
    gateway_name="DB",
    symbol=symbol,
    exchange=Exchange.BITMEX,
    datetime=now(),
    interval=Interval.MINUTE,
)
class TestDatabase(unittest.TestCase):

    def connect(self, settings: dict):
        from vnpy.trader.database.initialize import init  # noqa

        self.manager = init(settings)
    def test_save_load_bar(self):
        for driver, settings in profiles.items():
            with self.subTest(driver=driver, settings=settings):
                self.connect(settings)
                # save first
                self.manager.save_bar_data([bar])

                self.assertBarCount(43200, "there should be 43200 items after save")
TestDatabase(bar)